In [132]:
import os
import pandas as pd
import pandas
import numpy as np
import nltk
import gensim
import re
import string
import	matplotlib
import	matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score
from datetime import date
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora
%matplotlib	inline

In [4]:
# read data

CONST_WIKI_ALL = "/home/hady/ml/cscubs18/datasets/wiki_3classes2.csv"

dataset = np.genfromtxt(CONST_WIKI_ALL, delimiter="|\-/|", skip_header=1,
dtype={'names': ('klass', 'text'), 'formats': (np.int, '|S1000')})

docs = dataset['text']
labels = dataset['klass']

In [87]:
#fix the document
fixed = []

aux = 0
for line in docs:
    line = line.strip()
    line = line.decode('cp1252')
    fixed.extend([nltk.re.sub(r'[^\x00-\x7F]+', ' ', line)])


In [95]:
doc_complete = fixed
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]

doc_tfidf = []
for line in doc_clean:
    tmp = ""
    i = 0
    for word in line:
        tmp = tmp + word
        if i < (len(line) - 1):
            tmp = tmp + " "
    doc_tfidf.append(tmp)

In [98]:
feature_extraction = TfidfVectorizer()
data_iter = []
aux = 0
for t in doc_clean:
    data_iter.extend(t)
    
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(doc_tfidf)

(15000, 46542)


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=5)

In [134]:
clf = SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [161]:
predictions = clf.predict(X_test)
# print(predictions[:,1])
print('ROC-AUC yields ' + str(accuracy_score(y_test, predictions)))
print(clf.score(X_test, y_test))

# clf.predict(X_test)
# y = bin

ROC-AUC yields 0.33066666666666666
0.33066666666666666
